# 🛰️ 2D Scan Controller Demo

This notebook demonstrates the functionality of the **2D Scan Controller** implemented in Python.

Features demonstrated:
- Perform a full scan of the stage using `QD_Scan.scan_controller`
- Save the results (CSV)
- Visualize raw vs filtered sensor values
- Generate and display a heatmap of the scan data

The scan configuration is loaded from `config.yaml`.


In [9]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'QD_Scan'))
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import unittest
import logging
from unittest.mock import patch
from config import load_yaml_file
from functions import measure_sensor, sensor, stage, move_stage
from scan_controller import perform_scan, compute_rolling_average, detect_peak, generate_scan_points, save_to_csv
from visualize import csv_to_heatmap_array, plot_and_save_heatmap

## 🚀 Perform Full Scan

The scan will move the stage through the configured X/Y ranges.  
At each point, it will:
- Attempt to move the stage (with retry logic)
- Measure the sensor value (with retry logic)
- Apply a rolling-average filter
- Record raw and filtered values

Results are saved to CSV (`Scan_Data.csv`).


In [ ]:
config = load_yaml_file() #call yaml config
results, (peak_value, peak_point) = perform_scan(config, return_peak=True) #performs scan from scan_controller.py
peak_value, peak_point = detect_peak(results)
save_to_csv(results, config["output_csv"]) #saves results to csv
print(f"✅ Scan complete! Data saved to {config['output_csv']}")
#for row in results[:]: #to print all rows 
    #print(row)


## 📊 View Scan Data (Raw vs Filtered Values)

We will now load the scan results from CSV and visualize the **raw sensor values** vs **filtered (rolling average)** values.


In [ ]:
# Load CSV into DataFrame
df = pd.read_csv(config["output_csv"])
# Display first few rows
df.head()

In [ ]:
df = pd.read_csv(config["output_csv"])

# Keep only valid numeric rows
df = df[pd.to_numeric(df["filtered_value"], errors='coerce').notnull()]
# Plot raw vs filtered values
plt.figure(figsize=(10, 6))
#ax = plt.gca()
#ax.set_facecolor('black') 
plt.plot(df["raw_value"], label="Raw Value", marker='o')
plt.plot(df["filtered_value"], label="Filtered Value", marker='x')
plt.xlabel("Scan Point Index")
plt.ylabel("Sensor Value")
plt.title("Raw vs Filtered Sensor Values")
plt.legend()
plt.grid(True)
plt.show()


## 🗺️ Generate and Display Heatmap

We will now generate a **heatmap** from the filtered values to visualize spatial patterns in the scan data.


In [ ]:
# Convert CSV to heatmap array and plot
heatmap_array = csv_to_heatmap_array(
    filename=config["output_csv"],
    x_steps=config["x_range"]["steps"],
    y_steps=config["y_range"]["steps"]
)
plot_and_save_heatmap(heatmap_array, config["heatmap_output"], cmap="Reds", show_plot=True)

## 🌟 Peak Detection Result


In [ ]:
#Print and Append peak value to CSV
with open(config["output_csv"], mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([])  # empty row for separation
    writer.writerow(["Peak Value", "X", "Y"])
    if peak_value is not None:
        writer.writerow([peak_value, peak_point[0], peak_point[1]])
    else:
        writer.writerow(["None", "None", "None"])
if peak_value is not None:
    print(f"🌟 Global Peak Value: {peak_value} at Point: (X={peak_point[0]}, Y={peak_point[1]})")
else:
    print("⚠️ No valid peak detected.")


In [ ]:
# Plot raw vs filtered values with peak value highlighted
plt.figure(figsize=(10, 6))
#ax = plt.gca()
#ax.set_facecolor('black') 
plt.plot(df["raw_value"], color='blue',label="Raw Value", marker='o')
plt.plot(df["filtered_value"], color='orange', label="Filtered Value",marker='.')

# Highlight the peak on the filtered value plot
if peak_value is not None:
    # Find index of the peak point in the DataFrame
    peak_index = df[(df["x"] == peak_point[0]) & (df["y"] == peak_point[1])].index[0]
    
    # Plot marker on peak point
    plt.scatter(peak_index, peak_value, color='red', s=200, marker='*', label='Peak Point')

plt.xlabel("Scan Point Index")
plt.ylabel("Sensor Value")
plt.title("Raw vs Filtered Sensor Values (with Peak Highlighted)")
plt.legend()
plt.grid(True)
plt.show()
